**the goal of this notebook is to srape all articles which we can use as 'experts' to evaluate if the events in the hive are harmful**  
we will focus our attention to key words - 'bee' & 'acoustic' 

# load libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd
from random import randint
from time import sleep
#from selenium_stealth import stealth
import glob
import os

# Initiate the Chrome Driver

username and password for the website

In [2]:
username = "2239079053@shu.bg"
password = "BSru%SQ@trZz&nS26$cJ"

initiate the driver

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [15]:
import os
os.getcwd()

'/mnt/c/Users/Deni/PycharmProjects/AcousticData'

In [21]:
service = webdriver.FirefoxService(executable_path=r"geckodriver-v0.34.0-linux64/geckodriver.exe")
driver = webdriver.Firefox(service=service)

NoSuchDriverException: Message: Unable to locate or obtain driver for firefox; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


navigate to the research gate login website

In [6]:
driver.get("https://www.researchgate.net/login?_sg=ITbsht6C-ko8ZSF49e9deV1BNgFqKApIltdEguj_4uiZ9K_WzI6gYtnTL5xsgogphkn5Z2RJTNuYqd9fMiGJfg")

find the accept button

In [7]:
driver.find_element(By.CLASS_NAME, "css-1u05hh5").click()

add username and password 

In [7]:
uname = driver.find_element("id", "input-login") 
uname.send_keys(username)


In [8]:
pword = driver.find_element("id", "input-password") 
pword.send_keys(password)

click log in 

In [9]:
driver.find_element(By.XPATH, '//button[@type="submit"]').click()

# Search the key works

In [10]:
search = driver.find_element('name','query')

In [11]:
search.send_keys('bee acoustic')

In [ ]:
search.send_keys('bee acoustic machine learning')

In [12]:
driver.find_element(By.XPATH, '//button[@type="submit"]').click() #find button

click full text only 

In [13]:
driver.find_element(By.CLASS_NAME,'fulltext-availability-switch').click()

# Iterate over the search results 

after we arrived at the search page, we will need to iterate over all result pages and get the title,the url, the the authors, the date of publishing, the category, the type of the publication, the abstract and if the full text is available.  

In [15]:
df = pd.read_csv('bee_acoustic_scraped.csv')

In [17]:
df.tail()

,Title,URL,Authors,Date,Left Category,Right Category,Abstract,Footer,File Name,File
574,Complex Source Distributions in Wave-based Vir...,https://www.researchgate.net/publication/37329...,Stefan Bilbao,Aug 2023,Full-text available;Conference Paper,Internoise 2023,In the typical scenario in time-domain wave-ba...,;53 Reads,NaN,C:\Users\dpanova\Downloads\C000699.pdf
575,The Philippines stingless bee propolis promote...,https://www.researchgate.net/publication/36496...,Yulan Tang;Chen Wang;Mark Joseph Maranan Desam...,Oct 2022,Full-text available;Article,NaN,"Although hair loss is not a horrible disease, ...",;67 Reads;2 Citations,NaN,C:\Users\dpanova\Downloads\The_Philippines_sti...
576,Bee-associated fungi mediate effects of fungic...,https://www.researchgate.net/publication/35440...,Danielle Rutkowski;Eliza Litsey;Isabelle Maalo...,Sep 2021,File available;Preprint,NaN,Bumble bees are important pollinators that fac...,2 Recommendations;112 Reads;1 Citation,NaN,C:\Users\dpanova\Downloads\Bee-associated_fung...
577,Burzyan population of the dark forest honey be...,https://www.researchgate.net/publication/35753...,Rustem A. Ilyasov,Jan 2022,Full-text available;Article,NaN,"Ilyasov, R., 2021. Burzyan population of the d...",;137 Reads;1 Citation,NaN,C:\Users\dpanova\Downloads\Ilyasov_2021_Burzya...
578,A paediatric case of exercise-augmented anaphy...,https://www.researchgate.net/publication/36207...,Zhi Xiang Leang;Meera Thalayasingam;Michael O’...,Jul 2022,Full-text available;Article,NaN,Bee pollen is becoming an increasingly popular...,;5 Reads;2 Citations,NaN,C:\Users\dpanova\Downloads\Ilyasov_2021_Burzya...


In [17]:
# create a data frame which will store all of that information
#df = pd.DataFrame(columns = ['Title','URL','Authors','Date','Left Category','Right Category','Abstract', 'Footer','File Name'])

In [18]:
#we will loop overt the results of the first main page first 
result_items = driver.find_elements(By.CLASS_NAME,'search-box__result-item')

In [20]:
for result in result_items[8:]:
    #access the inner item for each result
    items = result.find_elements(By.CLASS_NAME, 'nova-legacy-v-publication-item__stack-item')
    
    #get the title from the first tile
    title = items[0].find_element(By.XPATH,'div//a').text 
    print(title)
    
    #get the url 
    url = items[0].find_element(By.XPATH,'div//a').get_attribute('href') 
    
    #get the left categories 
    temp_left = items[1].find_elements(By.CLASS_NAME,'nova-legacy-v-publication-item__meta-left')
    
    #since it may have more than one left category, we will iterate over it
    if len(temp_left) >1:
        left_list = []
        for i in range(len(temp_left)):
            left_list.append(temp_left[i].text)
        left = ';'.join(left_list)
    else: 
        left = temp_left[0].text
    
    #get the date 
    temp_right = items[1].find_elements(By.CLASS_NAME,'nova-legacy-v-publication-item__meta-right')
    date = temp_right[0].find_elements(By.XPATH, 'ul//li')[0].text
    
    #get the right category
    cat = temp_right[0].find_elements(By.XPATH, 'ul//li')
    if len(cat)==1:
        right = ''
    else: 
        right_list = []
        for i in range(1,len(cat)):
            right_list.append(cat[i].text)
        right = ';'.join(right_list)
    
    #get the authors 
    authors_list = items[2].find_elements(By.XPATH,'ul//li//a//span')
    if len(authors_list) >1:
        temp_authors_list = []
        for author in authors_list:
            if len(author.text)>0:
                temp_authors_list.append(author.text)
        author = ';'.join(temp_authors_list)
    else: 
        author = authors_list[0].text
    
    #get the abstract 
    abstract = items[3].text
    
    #get the footer 
    footer_box = result.find_element(By.CLASS_NAME,'nova-legacy-v-publication-item__footer-metrics')
    footer_list = []
    footer_items = footer_box.find_elements(By.XPATH,'ul//li')
    for f in footer_items: 
        footer_list.append(f.text)
    footer = ';'.join(footer_list)
    
    #download 
    #click the download button 
    result.find_elements(By.CLASS_NAME,'nova-legacy-c-button-group__item')[0].click()
    #find the file name
    sleep(randint(30,60))
    list_of_files = glob.glob('C:\\Users\\dpanova\\Downloads\\*')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    
    #append the results to the DF 
    page_dict = { 'Title':[title],
            'URL':[url],
            'Authors':[author],
            'Date':[date],
            'Left Category':[left],
            'Right Category':[right],
            'Abstract':[abstract],
            'Footer':[footer],
                 'File':[latest_file]
                 
        }
    page_df = pd.DataFrame(page_dict)
    df = pd.concat([df,page_df],ignore_index=True)

Nature‐inspired artificial bee colony‐based hyperparameter optimization of CNN for anomaly detection in induction motor
The Acoustic Emission in the Nest of the Honey Bee Depending on the Extreme Weather Conditions


## iterate over all 100 pages 

In [18]:
current_url = driver.current_url

In [20]:
main_url = current_url+ '&limit=10&offset='+ str(10)

In [21]:
driver.get(main_url)

In [22]:
result_items = driver.find_elements(By.CLASS_NAME,'search-box__result-item')

In [26]:
for step in range(770, 1000, 10):
    main_url = current_url+ '&limit=10&offset='+ str(step)
    driver.get(main_url)
    print(main_url)
    sleep(randint(10,30))
    
    #do the same steps
    result_items = driver.find_elements(By.CLASS_NAME,'search-box__result-item')
    for result in result_items:
        #access the inner item for each result
        items = result.find_elements(By.CLASS_NAME, 'nova-legacy-v-publication-item__stack-item')

        #get the title from the first tile
        title = items[0].find_element(By.XPATH,'div//a').text 
        print(title)

        #get the url 
        url = items[0].find_element(By.XPATH,'div//a').get_attribute('href') 

        #get the left categories 
        temp_left = items[1].find_elements(By.CLASS_NAME,'nova-legacy-v-publication-item__meta-left')

        #since it may have more than one left category, we will iterate over it
        if len(temp_left) >1:
            left_list = []
            for i in range(len(temp_left)):
                left_list.append(temp_left[i].text)
            left = ';'.join(left_list)
        else: 
            left = temp_left[0].text

        #get the date 
        temp_right = items[1].find_elements(By.CLASS_NAME,'nova-legacy-v-publication-item__meta-right')
        date = temp_right[0].find_elements(By.XPATH, 'ul//li')[0].text

        #get the right category
        cat = temp_right[0].find_elements(By.XPATH, 'ul//li')
        if len(cat)==1:
            right = ''
        else: 
            right_list = []
            for i in range(1,len(cat)):
                right_list.append(cat[i].text)
            right = ';'.join(right_list)

        #get the authors 
        authors_list = items[2].find_elements(By.XPATH,'ul//li//a//span')
        if len(authors_list) >1:
            temp_authors_list = []
            for author in authors_list:
                if len(author.text)>0:
                    temp_authors_list.append(author.text)
            author = ';'.join(temp_authors_list)
        else: 
            try: 
                author = authors_list[0].text
            except: 
                author = ''

        #get the abstract 
        abstract = items[3].text

        #get the footer 
        footer_box = result.find_element(By.CLASS_NAME,'nova-legacy-v-publication-item__footer-metrics')
        footer_list = []
        footer_items = footer_box.find_elements(By.XPATH,'ul//li')
        for f in footer_items: 
            footer_list.append(f.text)
        footer = ';'.join(footer_list)

        #download 
        #click the download button 
        result.find_elements(By.CLASS_NAME,'nova-legacy-c-button-group__item')[0].click()
        #find the file name
        sleep(randint(30,60))
        list_of_files = glob.glob('C:\\Users\\dpanova\\Downloads\\*')
        latest_file = max(list_of_files, key=os.path.getctime)
        
        
        #click 'no thanks' button
        try: 
            buttons = driver.find_elements(By.XPATH, '//button')
            for b in buttons: 
                if b.text == 'No thanks':
                    b.click()
                    latest_file=None
        except: 
            None 
            

        #append the results to the DF 
        page_dict = { 'Title':[title],
                'URL':[url],
                'Authors':[author],
                'Date':[date],
                'Left Category':[left],
                'Right Category':[right],
                'Abstract':[abstract],
                'Footer':[footer],
                     'File':[latest_file]

            }
        page_df = pd.DataFrame(page_dict)
        df = pd.concat([df,page_df],ignore_index=True)

#before 930 >> 9,10



https://www.researchgate.net/search.Search.html?query=bee+acoustic&type=publication&subfilter%5Bfulltext%5D=ft&limit=10&offset=770
EFFECT OF GLOBAL ACCELERATION IN THE ACOUSTIC RESPONSE OF PREMIXED FLAME AND ANALYSIS OF ACOUSTIC INSTABILITY OF COMBUSTION CHAMBER
A global review of determinants of native bee assemblages in urbanised landscapes
Effect of a modulated acoustic field on the dynamics of a vibrating charged bubble
Channel Emulator Framework for Underwater Acoustic Communications
Assessment of Environmental Acoustic Levels of Some Churches in Uyo Metropolis, Nigeria
"The effects of supplementary pollen diets on population, growth, and performance of honey bee "
Acoustic Signals and Behavior of the Invasive Freshwater Crayfish Cherax destructor (Clark, 1936)
Relationship among five‐factor personality traits and psychological distress with acoustic analysis
Exposure to sublethal concentration of flupyradifurone alters sexual behavior and cuticular hydrocarbon profile in Heriades

Разходите във връзка с производството на пчелни продукти -финансово и управленско счетоводно третиране The costs of the production of bee products -financial and management accounting treatment
The reliability of honey bee density estimates from trapped drones
Current Knowledge on Bee Innate Immunity Based on Genomics and Transcriptomics
High-throughput fabrication of cell spheroids with 3D acoustic assembly devices
https://www.researchgate.net/search.Search.html?query=bee+acoustic&type=publication&subfilter%5Bfulltext%5D=ft&limit=10&offset=850
Bee species diversity and nesting sites in cultivated savannah, northern Zimbabwe
Increased Efficiency of the Artificial Bee Colony Algorithm Using the Pheromone Technique
Research and Implementation on a Real-time OSDM MODEM for Underwater Acoustic Communications
A noise-robust acoustic method for recognition of foraging activities of grazing cattle
Hybrid Machine Learning Framework for Multistage Parkinson’s Disease Classification Using Acoust

Bee Venom Induces Acute Inflammation through a H2O2-Mediated System That Utilizes Superoxide Dismutase
https://www.researchgate.net/search.Search.html?query=bee+acoustic&type=publication&subfilter%5Bfulltext%5D=ft&limit=10&offset=920
The local environmental factors impact the infestation of bee colonies by mite Varroa destructor
Seven new records of bee flies (Bombyliidae, Diptera) from Saudi Arabia
A Machine Learning Approach to Qualitatively Evaluate Different Granulation Phases by Acoustic Emissions
A Method for Degenerate Primer Design Based on Artificial Bee Colony Algorithm
Investigating overlapping gestures in acoustic signals: The case of fricatives and sonorants in clusters
Stingless bee propolis, metformin, and their combination alleviate diabetic cardiomyopathy
Flexible control of vocal timing in bats enables escape from acoustic interference
Observation of thermal acoustic modes of a droplet coupled to an optomechanical sensor
Chemical Profile and Antioxidant Capacity of Pr

3D underwater acoustic Luneburg lens based on gradient face-centered-cubic phononic crystals
Ultra-broadband illusion acoustics for space and time camouflages
Efficacy of Saudi propolis and bee pollen in the reduction of oxidative stress induced with CCl4 in a testis mice model
Damage Detection and Optimization of Concrete Structure Based on Acoustic Emission Technology
Dialect classification using acoustic and linguistic features in Arabic speech
Evaluating the potential for mating control in honey bee breeding in three SE European countries (preliminary results)
A brief description on AIR COUPLED ACOUSTIC THERMOGRAPHY (ACAT) INSPECTION TECHNIQUE
Genetic Variation in Antimicrobial Activity of Honey Bee (Apis mellifera) Seminal Fluid
Bee stimulation to form protein food reserves


In [29]:
len(df)

960

In [31]:
len(df['Title'].unique())

893

In [33]:
df.to_excel('bee_acoustic_scraped.xlsx',index=False)

In [32]:
df.to_csv('bee_acoustic_scraped.csv',index=False)